In [ ]:
%load_ext lab_black

In [ ]:
experiment_id = 0

In [ ]:
"""Setup the environment."""
from pathlib import Path
import os
import sys

import numpy as np
from numpy.typing import ArrayLike
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, Markdown, HTML

# Add utility package to path
sys.path.append(os.environ.get("BRAINWEB_TDCS_CODE_DIR", "../code"))
from brainweb_tdcs import ROIS, EXPERIMENTS
from brainweb_tdcs.study import get_experiments_for_roi
from brainweb_tdcs.plot import plot_density

# Set path data directory
if "BRAINWEB_TDCS_DATA_DIR" not in os.environ:
    os.environ["BRAINWEB_TDCS_DATA_DIR"] = str((Path.cwd() / "../data").resolve())

In [ ]:
"""Select the experiment."""
experiment = EXPERIMENTS[experiment_id]
display(
    Markdown(
        "# Compute the induced transmembrane potential ratio to size "
        f"({experiment.roi}, {experiment.montage} electrodes montage)"
    )
)

In [ ]:
display(
    HTML(
        f"""
<div style="background: #efefef; color: #5f5f5f; padding: 15pt;">
    <b style="display: inline-block; width: 120pt;">Region of interest</b> {experiment.roi.long_name} ({experiment.roi})</br>
    <b style="display: inline-block; width: 120pt;">Electrodes montage</b> {experiment.montage} ({"Bipolar" if experiment.is_bipolar else "Unipolar"})</br>
    </br>
    In this notebook, we compute the induced transmembrane potential in a theoretical cell of the region of interest resulting from the stimulation.
</div>
"""
    )
)

## Data

First, we load the data corresponding to this experiment and display the probability density of the mean absolute magnitude of the electric field $\bar{| \pmb{e} |}$ (mVm$^{-1}$) recorded in the region of interest during the simulations.

In [ ]:
"""Load the experiment results."""
e, e_t, e_r = (
    experiment.get_data()[["e", "e_t", "e_r"]].values[:, :, np.newaxis].swapaxes(0, 1)
)

In [ ]:
"""Plot mean absolute magnitude of the electric field."""
fig, ax = plt.subplots(1, 1, figsize=(10, 5))
plot_density(
    ax,
    pd.Series(e.ravel()),
    experiment,
    "$| \\bar{\mathbf{e}} |$ (mVm$^{-1}$)",
    (
        "Probability density of the mean absolute magnitude of the electric field\n"
        f"computed in the {experiment.roi} ({experiment.montage} electrodes montage)"
    ),
)
plt.show()
pd.Series(e.ravel()).describe().to_frame().round(3).transpose()[
    ["mean", "std", "min", "25%", "75%", "max"]
]

## Spherical cell

The steady-state induced transmembrane potential $\Delta u_\text{i}$ (V) on a spherical cell of radius $r_1$ (m) with a non-conductive plasma membrane induced by an external electric field $\pmb{e}$ (V/m) is described by Schwan's equation (Schwan, 1957)
$$
\Delta u_\text{i} = \frac{3}{2} | \pmb{e} | r_1 \text{cos}(\theta),
$$
where $\theta$ is the angle between the electric field and the vector going from the center of the cell to the point where $\Delta u_\text{i}$ is calculated.

### Maximum value of $\Delta u_\text{i}$

For this expression of $\Delta u_\text{i}$, it is easy to determine the maximum value since it is obtained when $\theta = 0$. Thus,
$$
\text{max}( \Delta u_\text{i} ) = \frac{3}{2} | \pmb{e} | r_1.
$$

### Maximum value of $\Delta u_\text{i} / r_1$

The previous equation can be transformed to determine a ratio between the induced transmembrane potential and the radius of the cell
$$
\frac{\text{max}( \Delta u_\text{i} )}{r_1} = \frac{3}{2} | \pmb{e} |.
$$
The ratio is thus independent of the size of the cell and is more informative. To find the maximum induced transmembrane potential, one simply multiplies the results by the size of the cell of interest.

In [ ]:
"""Compute induced transmembrane potential to size ratio for spherical cells."""
Δu_i_sphere = (3 / 2) * e

In [ ]:
"""Plot induced transmembrane potential to size ratio for a spherical cell."""
fig, ax = plt.subplots(1, 1, figsize=(10, 5))
plot_density(
    ax,
    pd.Series(Δu_i_sphere.ravel()),
    experiment,
    "$\Delta u_{i} / r_1$ (mVm$^{-1}$)",
    (
        "Induced transmembrane potential to size ratio for a spherical cell\n"
        f"computed in the {experiment.roi} ({experiment.montage} electrodes montage)"
    ),
)
plt.show()
summary = (
    pd.Series(Δu_i_sphere.ravel())
    .describe()
    .to_frame()
    .transpose()[["mean", "std", "min", "25%", "75%", "max"]]
).round(3)
summary.index = ["Δu_i"]
summary

## Spheroidal cell

Now, let us consider a prolate spheroidal cell ($r_1 > r_2 = r_3$) with a shape ratio $\gamma = r_1 / r_2$. The definition of the induced transmembrane potential for such a cell (Valic *et al.*, 2003) is
$$
\Delta u_\text{i} = | \pmb{e} | \text{sin}(\theta) \frac{x}{1 - l_x} + | \pmb{e} | \text{cos}(\theta) \frac{z}{1 - l_z},
$$
where $x$ and $z$ define the coordinates of the point where $\Delta u_\text{i}$ is computed on the surface and $\theta$ is the angle between the vector going from the center of the cell to the point. The depolarisation factors $l_x$ and $l_z$ depend on the shape of the cell and are given by
$$
\begin{split}
    l_x = l_y &= \frac{1}{2} (1 - l_z), \\
    l_z &= \frac{1 - \lambda^2}{2 \lambda^3} \left( \text{log}\left( \frac{1 + \lambda}{1 - \lambda} \right) - 2 \lambda \right),
\end{split}
$$
where $\lambda = \sqrt{1 - (r_2 / r_1)^2}$.

For this work, we consider that the spheroidal cells are elongated along the normal of the cortical surface. This allow us to write
$$
\begin{split}
    | \pmb{e} | \text{sin}(\theta) &= | \pmb{e}_t |, \\
    | \pmb{e} | \text{cos}(\theta) &= | \pmb{e}_r |,
\end{split}
$$
where $| \pmb{e}_t |$ and $| \pmb{e}_r |$ are respectively referred to as the tangential and radial components of the electric field with regard to the cortical surface. From the expression of an ellipse, we also have
$$
\begin{split}
    x &= r_2 \text{sin}(\varphi), \\
    z &= r_1 \text{cos}(\varphi),
\end{split}
$$
in which $\varphi$ is the angle between the main axis of the ellipse and the point where $\Delta u_\text{i}$ is calculated. Finally, the expression of the induced transmembrane potential becomes
$$
\Delta u_\text{i} = | \pmb{e}_t | \frac{r_2 \text{sin}(\varphi)}{1 - l_x} + | \pmb{e}_r | \frac{r_1 \text{cos}(\varphi)}{1 - l_z}.
$$

### Maximum value of $\Delta u_\text{i}$

Since we are only interested in the maximum possible value of $\Delta u_\text{i}$, we define the first derivative of the previous equation with regard to $\varphi$ and get
$$
\frac{\partial}{\partial \varphi}\Delta u_\text{i} = -| \pmb{e}_t | \frac{r_2 \text{cos}(\varphi)}{1 - l_x} + | \pmb{e}_r | \frac{r_1 \text{sin}(\varphi)}{1 - l_z}.
$$
It is equal to $0$ when $$
\varphi \pm k \pi
= \text{atan}\left( \frac{| \pmb{e}_t | r_2 (1 - l_z)}{| \pmb{e}_r | r_1 (1 - l_x)} \right)
= \text{atan}\left( \frac{| \pmb{e}_t | (1 - l_z)}{| \pmb{e}_r | \gamma (1 - l_x)} \right)
= \text{atan}\left( \eta \frac{| \pmb{e}_t |}{| \pmb{e}_r |} \right),
$$
thus we only compute this value.

### Maximum value of $\Delta u_\text{i} / r_1$

Once again, we define the expression of the ratio between the induced transmembrane potential and the size of the cell. It is possible because $r_1$ and $r_2$ are linked together by $\gamma$. Consequently, we have
$$
\Delta u_\text{i}
= | \pmb{e}_t | \frac{r_1 \text{sin}(\varphi)}{\gamma (1 - l_x)} + | \pmb{e}_r | \frac{r_1 \text{cos}(\varphi)}{1 - l_z}
= r_1 \left( | \pmb{e}_t | \frac{ \text{sin}(\varphi)}{\gamma (1 - l_x)} + | \pmb{e}_r | \frac{\text{cos}(\varphi)}{1 - l_z} \right),
$$
which leads to the ratio we are interested in
$$
\frac{\Delta u_\text{i}}{r_1} = | \pmb{e}_t | \frac{ \text{sin}(\varphi)}{\gamma (1 - l_x)} + | \pmb{e}_r | \frac{\text{cos}(\varphi)}{1 - l_z}.
$$

In [ ]:
"""Define shape ratios."""
γ = np.array([10 / 8, 10 / 5, 10 / 2])

In [ ]:
"""Compute required parameters."""
γ = γ[np.newaxis, :]

In [ ]:
"""Compute depolarization factors."""
λ = np.sqrt(1 - (1 / γ) ** 2)
l_z = ((1 - λ**2) / (2 * λ**3)) * (np.log((1 + λ) / (1 - λ)) - 2 * λ)
l_x = (1 - l_z) / 2

In [ ]:
"""Compute angle maximizing the induced transmembrane potential."""
η = (1 - l_z) / (γ * (1 - l_x))
φ = np.arctan(η * e_t / e_r)

In [ ]:
"""Compute induced transmembrane potential to size ratio for spheroidal cells."""
Δu_i_spheroid = e_t / γ * np.sin(φ) * (1 - l_x) + e_r * np.cos(φ) * (1 - l_z)

In [ ]:
"""Plot induced transmembrane potential to size ratio for a spheroidal cell."""
fig, ax = plt.subplots(1, 1, figsize=(10, 5))
plot_density(
    ax,
    pd.Series(Δu_i_spheroid.ravel()),
    experiment,
    "$\Delta u_{i} / r_1$ (mVm$^{-1}$)",
    (
        "Induced transmembrane potential to size ratio for a spheroidal cell\n"
        f"computed in the {experiment.roi} ({experiment.montage} electrodes montage)"
    ),
)
plt.show()
summary = (
    pd.DataFrame(Δu_i_spheroid)
    .describe()
    .transpose()[["mean", "std", "min", "25%", "75%", "max"]]
).round(3)
summary.index = [f"Δu_i (γ={gamma})" for gamma in ("10/8", "10/5", "10/2")]
summary

## References

- Schwan (1957) - https://doi.org/10.1016/B978-1-4832-3111-2.50008-0
- Valic *et al.* (2003) - https://doi.org/10.1007/s00249-003-0296-9